In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
file_path = "/content/drive/My Drive/yelp_training_set_flattened.csv"
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,business_blank,business_categories,business_city,business_full_address,business_id,business_latitude,business_longitude,business_name,business_neighborhoods,...,reviewer_funny,reviewer_name,reviewer_review_count,reviewer_type,reviewer_useful,stars,text,type,useful,user_id
0,0,False,Breakfast & Brunch; Restaurants,Phoenix,"6106 S 32nd St\nPhoenix, AZ 85042",9yKzy9PApeiPPOUJEtnvkg,33.390792,-112.012504,Morning Glory Cafe,NaN,...,331,Jason,376,user,1034,5,My wife took me here on my birthday for breakf...,review,5,rLtl8ZkDX5vH5nAx9C3q5Q
1,1,False,Italian; Pizza; Restaurants,Phoenix,"4848 E Chandler Blvd\nPhoenix, AZ 85044",ZRJwVLyzEJq1VAihDhYiow,33.305607,-111.978758,Spinato's Pizzeria,NaN,...,2,Paul,2,user,0,5,I have no idea why some people give bad review...,review,0,0a2KyEL0d3Yb1V6aivbIuQ
2,2,False,Middle Eastern; Restaurants,Tempe,"1513 E Apache Blvd\nTempe, AZ 85281",6oRAC4uyJCsJl1X0WZpVSA,33.414345,-111.913031,Haji-Baba,NaN,...,0,Nicole,3,user,3,4,love the gyro plate. Rice is so good and I als...,review,1,0hT2KtfLiobPvh6cDC8JQg
3,3,False,Active Life; Dog Parks; Parks,Scottsdale,"5401 N Hayden Rd\nScottsdale, AZ 85250",_1QQZuf4zZOyFCvXc0o6Vg,33.522945,-111.907886,Chaparral Dog Park,NaN,...,18,lindsey,31,user,75,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,2,uZetl9T0NcROGOyFfughhg
4,4,False,Tires; Automotive,Mesa,"1357 S Power Road\nMesa, AZ 85206",6ozycU1RpktNG2-1BroVtw,33.391027,-111.684482,Discount Tire,NaN,...,3,Roger,28,user,32,5,General Manager Scott Petello is a good egg!!!...,review,0,vYmM4KTsC8ZfQBg-j5MWkw


Preprocessing

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
def preprocessing(text):
  if not isinstance(text, str):
        return ""
  tokens = word_tokenize(text)
  pos_tags = pos_tag(tokens)
  cleaned_tokens = []

  for word,tag in pos_tags:
     if word.lower() not in stop_words and word not in string.punctuation:
            wordnet_pos = get_wordnet_pos(tag)
            lemmatized_word = lemmatizer.lemmatize(word.lower(), wordnet_pos)
            cleaned_tokens.append(lemmatized_word)
  return " ".join(cleaned_tokens)

In [ ]:
df_sampled = df[:2500]
df_sampled['text'] = df_sampled['text'].apply(preprocessing)

<ipython-input-9-6540236a4f0f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sampled['text'] = df_sampled['text'].apply(preprocessing)


In [ ]:
df_sampled['text'].head()

,text
0,wife take birthday breakfast excellent weather...
1,idea people give bad review place go show plea...
2,love gyro plate rice good also dig candy selec...
3,rosie dakota love chaparral dog park 's conven...
4,general manager scott petello good egg go deta...


Sentimental Scores with BERT-base-uncased Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def get_sentiment_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    scores = torch.nn.functional.softmax(outputs.logits, dim=1)
    sentiment_score = torch.argmax(scores).item()

    return sentiment_score

df_sampled['sentimental score'] = df_sampled['text'].apply(get_sentiment_score)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

<ipython-input-11-63148eab65de>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sampled['sentimental score'] = df_sampled['text'].apply(get_sentiment_score)


In [ ]:
df_sampled['sentimental score'].describe()

,sentimental score
count,2500.000000
mean,2.528800
std,1.517001
min,0.000000
25%,1.000000
50%,3.000000
75%,4.000000
max,4.000000


In [ ]:
df_sampled['sentimental score']

,sentimental score
0,4
1,0
2,4
3,4
4,3
...,...
2495,4
2496,0
2497,4
2498,4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229907 entries, 0 to 229906
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              229907 non-null  int64  
 1   business_blank          229907 non-null  bool   
 2   business_categories     229130 non-null  object 
 3   business_city           229907 non-null  object 
 4   business_full_address   229907 non-null  object 
 5   business_id             229907 non-null  object 
 6   business_latitude       229907 non-null  float64
 7   business_longitude      229907 non-null  float64
 8   business_name           229907 non-null  object 
 9   business_neighborhoods  0 non-null       float64
 10  business_open           229907 non-null  bool   
 11  business_review_count   229907 non-null  int64  
 12  business_stars          229907 non-null  float64
 13  business_state          229907 non-null  object 
 14  business_type       

In [ ]:
restaurant_keywords = ['Restaurant']
hotel_keywords = ['Hotel']

df_sampled['Type'] = df_sampled['business_categories'].apply(
    lambda x: 'Restaurant' if any(word in str(x) for word in restaurant_keywords)
    else ('Hotel' if any(word in str(x) for word in hotel_keywords) else 'Other')
)

restaurant_df = df_sampled[df_sampled['Type'] == 'Restaurant'].reset_index(drop=True)
hotel_df = df_sampled[df_sampled['Type'] == 'Hotel'].reset_index(drop=True)


<ipython-input-15-0904233a2160>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sampled['Type'] = df_sampled['business_categories'].apply(


Deep Matrix Factorization Predictions

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


def train_deep_mf(df, embedding_dim=8, epochs=20, batch_size=50, validation_split=0.1):

    user_ids = df['review_id'].astype('category').cat.codes.values
    item_ids = df['business_id'].astype('category').cat.codes.values
    ratings = df['stars'].values

    num_users = len(np.unique(user_ids))
    num_items = len(np.unique(item_ids))

    user_input = keras.layers.Input(shape=(1,))
    item_input = keras.layers.Input(shape=(1,))
    user_embedding = keras.layers.Embedding(num_users, embedding_dim, input_length=1)(user_input)
    item_embedding = keras.layers.Embedding(num_items, embedding_dim, input_length=1)(item_input)

    user_vec = keras.layers.Flatten()(user_embedding)
    item_vec = keras.layers.Flatten()(item_embedding)
    concat = keras.layers.Concatenate()([user_vec, item_vec])
    dense1 = keras.layers.Dense(32, activation='relu')(concat)
    dense2 = keras.layers.Dense(16, activation='relu')(dense1)
    output = keras.layers.Dense(1)(dense2)

    model = keras.models.Model(inputs=[user_input, item_input], outputs=output)
    model.compile(optimizer='adam', loss='mse')

    model.fit([user_ids, item_ids], ratings, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

    df['DeepMF_predictions'] = model.predict([user_ids, item_ids]).flatten()

    return df

In [ ]:
train_deep_mf(restaurant_df)
train_deep_mf(hotel_df)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 14.6656 - val_loss: 13.7693
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 12.4240 - val_loss: 10.1726
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.2764 - val_loss: 4.8855
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.9132 - val_loss: 1.7439
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.0087 - val_loss: 1.6360
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6332 - val_loss: 1.7019
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3242 - val_loss: 1.7737
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1353 - val_loss: 1.8439
Epoch 9/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0536 - val_loss: 1.8783
Epoch 10/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0192 - val_loss: 1.9080
Epoch 11/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0088 - val_loss: 1.9187
Epoch 12/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0029 - val_loss: 1.

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 297ms/step - loss: 15.1577 - val_loss: 17.7111
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 14.6355 - val_loss: 17.6088
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 14.6272 - val_loss: 17.5108
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 14.4085 - val_loss: 17.4156
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 14.6922 - val_loss: 17.3231
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 14.6011 - val_loss: 17.2361
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 14.7509 - val_loss: 17.1471
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 14.6362 - val_loss: 17.0563
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 14.6300 - val_loss: 16.9635
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 14.3119 - val_loss: 16.8629
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 14.2675 - val_loss: 16.7570
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 14.3297 - val_l

,Unnamed: 0,business_blank,business_categories,business_city,business_full_address,business_id,business_latitude,business_longitude,business_name,business_neighborhoods,...,stars,text,type,useful,user_id,sentimental score,Type,DeepMF_predictions,text_vector,cluster
0,7,False,Hotels & Travel; Airports,Phoenix,"3400 E Sky Harbor Blvd\nPhoenix, AZ 85034",hW0Ne_HTHEAgGF1rAdmR-g,33.434750,-112.006440,Phoenix Sky Harbor International Airport,NaN,...,4,luckily n't travel far make connect flight tha...,review,1,1ieuYcKS7zeAv_U15AB13A,2,Hotel,0.307030,"[-0.015138562, 0.02942112, 0.0017445359, -0.00...",4
1,19,False,Hotels & Travel; Airlines; Transportation,Phoenix,Phoenix Sky Harbor Airport\nTerminal 3\nPhoeni...,4JzzbSbK9wmlOBJZWYfuCg,33.439648,-112.026154,Frontier Airlines,NaN,...,4,'ve get well good time since review write last...,review,1,nDBly08j5URmrHQ2JCbyiw,1,Hotel,0.317520,"[-0.012675355, 0.026620658, 0.0030172463, -0.0...",1
2,83,False,Hotels & Travel; Event Planning & Services; Ho...,Phoenix,"401 W Clarendon Ave\nPhoenix, AZ 85013",8ZwO9VuLDWJOXmtAdc7LXQ,33.491120,-112.079081,Clarendon Hotel,NaN,...,4,great hotel central phoenix stay-cation necess...,review,0,qLCpuCWCyPb4G2vN-WZz-Q,4,Hotel,0.390176,"[-0.024094759, 0.039820373, 0.0056840316, -0.0...",0
3,90,False,Hotels & Travel; Airports,Phoenix,"3400 E Sky Harbor Blvd\nPhoenix, AZ 85034",hW0Ne_HTHEAgGF1rAdmR-g,33.434750,-112.006440,Phoenix Sky Harbor International Airport,NaN,...,1,beach paradise ixtapa-zihuatenejo await u .......,review,7,8tbXmjYGsYFZXk6ppuwRWQ,0,Hotel,0.306639,"[-0.010430062, 0.023214886, 0.0012226121, -0.0...",3
4,129,False,Hotels & Travel; Event Planning & Services; Ho...,Tempe,"1221 E Apache Boulevard\nTempe, AZ 85281",c0iszTWZwYtO3TgBx0Z0fQ,33.414027,-111.918234,Days Inn Tempe-Asu,NaN,...,2,feel day inn tempe best describe `` place purc...,review,3,rVlgz-MGYRPa8UzTYO0RGQ,0,Hotel,0.313501,"[-0.016071603, 0.030229472, 0.003476869, -0.00...",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,2320,False,Hotels & Travel; Event Planning & Services; Ho...,Tempe,"951 East Playa Del Norte Drive\nTempe, AZ 85281",OkxO2FoKMxVCFzskLeI37Q,33.434736,-111.924312,Aloft Tempe,NaN,...,4,stay group friend 1/2 marathon register room f...,review,0,gw2sxzQrSTo6NpMSp_gVNQ,3,Hotel,0.280189,"[-0.01642188, 0.033307947, 0.0033832588, -0.00...",2
77,2452,False,Hotels & Travel; Event Planning & Services; Ho...,Phoenix,"340 North 3rd Street\nPhoenix, AZ 85004",FgyBmc-XS_ViFAf8Wy25yA,33.452693,-112.070113,Sheraton Phoenix Downtown Hotel,NaN,...,5,recently stay sheraton two night attend confer...,review,1,El-nrOv6QtsmsOnkXQDknQ,4,Hotel,0.231392,"[-0.021716733, 0.04076336, 0.0045391354, -0.00...",0
78,2460,False,Hotels & Travel; Airports,Phoenix,"3400 E Sky Harbor Blvd\nPhoenix, AZ 85034",hW0Ne_HTHEAgGF1rAdmR-g,33.434750,-112.006440,Phoenix Sky Harbor International Airport,NaN,...,1,rental car review 118 degree stand outside hea...,review,1,0EtZMnRQF2YsXvWBQ7FauQ,0,Hotel,0.277814,"[-0.014800081, 0.029110746, 0.002967948, -0.00...",3
79,2468,False,Hotels & Travel; Event Planning & Services; Ho...,Scottsdale,"6850 E Main St\nScottsdale, AZ 85251",6Kon3cR5ZEm5rmYKlpcfcw,33.493051,-111.933956,Hotel Valley Ho,NaN,...,5,stay one night travel phoenix business cool ho...,review,1,JJnS30Cjv5CBa1JXyZU5yw,4,Hotel,0.336736,"[-0.025511362, 0.049761504, 0.0054899673, -0.0...",0


In [ ]:
restaurant_df['DeepMF_predictions']

,DeepMF_predictions
0,4.999794
1,4.999228
2,3.996981
3,4.003074
4,5.003398
...,...
1743,3.332404
1744,3.657959
1745,2.708824
1746,2.815094


In [ ]:
hotel_df['DeepMF_predictions']

,DeepMF_predictions
0,0.307030
1,0.317520
2,0.390176
3,0.306639
4,0.313501
...,...
76,0.280189
77,0.231392
78,0.277814
79,0.336736


Clusttering

In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering

def kmeans_clustering(df, text_column='text', sentiment_column='sentimental score',
                            vector_size=100, window=5, min_count=1, workers=4,
                            pca_components=20, num_clusters=5):

    texts = df[text_column].astype(str).apply(lambda x: x.split())

    w2v_model = Word2Vec(sentences=texts, vector_size=vector_size, window=window, sg=1,
                         min_count=min_count, workers=workers)

    def text_to_vector(tokens):
        vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
        return np.mean(vectors, axis=0) if vectors else np.zeros(vector_size)

    df['text_vector'] = df[text_column].apply(lambda x: text_to_vector(x.split()))

    text_feature_matrix = np.vstack(df['text_vector'])
    text_feature_df = pd.DataFrame(text_feature_matrix, columns=[f'w2v_{i}' for i in range(vector_size)])

    numerical_features = df[[sentiment_column]]
    scaler = StandardScaler()
    numerical_features_scaled = scaler.fit_transform(numerical_features)
    numerical_df = pd.DataFrame(numerical_features_scaled, columns=[sentiment_column])

    final_features = pd.concat([text_feature_df, numerical_df], axis=1)

    pca = PCA(n_components=pca_components)
    final_features_pca = pca.fit_transform(final_features)

    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    df['cluster'] = kmeans.fit_predict(final_features_pca)

    return df


In [ ]:
kmeans_clustering(restaurant_df)
kmeans_clustering(hotel_df)

,Unnamed: 0,business_blank,business_categories,business_city,business_full_address,business_id,business_latitude,business_longitude,business_name,business_neighborhoods,...,stars,text,type,useful,user_id,sentimental score,Type,DeepMF_predictions,text_vector,cluster
0,7,False,Hotels & Travel; Airports,Phoenix,"3400 E Sky Harbor Blvd\nPhoenix, AZ 85034",hW0Ne_HTHEAgGF1rAdmR-g,33.434750,-112.006440,Phoenix Sky Harbor International Airport,NaN,...,4,luckily n't travel far make connect flight tha...,review,1,1ieuYcKS7zeAv_U15AB13A,2,Hotel,0.307030,"[-0.015138562, 0.02942112, 0.0017445359, -0.00...",4
1,19,False,Hotels & Travel; Airlines; Transportation,Phoenix,Phoenix Sky Harbor Airport\nTerminal 3\nPhoeni...,4JzzbSbK9wmlOBJZWYfuCg,33.439648,-112.026154,Frontier Airlines,NaN,...,4,'ve get well good time since review write last...,review,1,nDBly08j5URmrHQ2JCbyiw,1,Hotel,0.317520,"[-0.012675355, 0.026620658, 0.0030172463, -0.0...",1
2,83,False,Hotels & Travel; Event Planning & Services; Ho...,Phoenix,"401 W Clarendon Ave\nPhoenix, AZ 85013",8ZwO9VuLDWJOXmtAdc7LXQ,33.491120,-112.079081,Clarendon Hotel,NaN,...,4,great hotel central phoenix stay-cation necess...,review,0,qLCpuCWCyPb4G2vN-WZz-Q,4,Hotel,0.390176,"[-0.024094759, 0.039820373, 0.0056840316, -0.0...",0
3,90,False,Hotels & Travel; Airports,Phoenix,"3400 E Sky Harbor Blvd\nPhoenix, AZ 85034",hW0Ne_HTHEAgGF1rAdmR-g,33.434750,-112.006440,Phoenix Sky Harbor International Airport,NaN,...,1,beach paradise ixtapa-zihuatenejo await u .......,review,7,8tbXmjYGsYFZXk6ppuwRWQ,0,Hotel,0.306639,"[-0.010430062, 0.023214886, 0.0012226121, -0.0...",3
4,129,False,Hotels & Travel; Event Planning & Services; Ho...,Tempe,"1221 E Apache Boulevard\nTempe, AZ 85281",c0iszTWZwYtO3TgBx0Z0fQ,33.414027,-111.918234,Days Inn Tempe-Asu,NaN,...,2,feel day inn tempe best describe `` place purc...,review,3,rVlgz-MGYRPa8UzTYO0RGQ,0,Hotel,0.313501,"[-0.016071603, 0.030229472, 0.003476869, -0.00...",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,2320,False,Hotels & Travel; Event Planning & Services; Ho...,Tempe,"951 East Playa Del Norte Drive\nTempe, AZ 85281",OkxO2FoKMxVCFzskLeI37Q,33.434736,-111.924312,Aloft Tempe,NaN,...,4,stay group friend 1/2 marathon register room f...,review,0,gw2sxzQrSTo6NpMSp_gVNQ,3,Hotel,0.280189,"[-0.01642188, 0.033307947, 0.0033832588, -0.00...",2
77,2452,False,Hotels & Travel; Event Planning & Services; Ho...,Phoenix,"340 North 3rd Street\nPhoenix, AZ 85004",FgyBmc-XS_ViFAf8Wy25yA,33.452693,-112.070113,Sheraton Phoenix Downtown Hotel,NaN,...,5,recently stay sheraton two night attend confer...,review,1,El-nrOv6QtsmsOnkXQDknQ,4,Hotel,0.231392,"[-0.021716733, 0.04076336, 0.0045391354, -0.00...",0
78,2460,False,Hotels & Travel; Airports,Phoenix,"3400 E Sky Harbor Blvd\nPhoenix, AZ 85034",hW0Ne_HTHEAgGF1rAdmR-g,33.434750,-112.006440,Phoenix Sky Harbor International Airport,NaN,...,1,rental car review 118 degree stand outside hea...,review,1,0EtZMnRQF2YsXvWBQ7FauQ,0,Hotel,0.277814,"[-0.014800081, 0.029110746, 0.002967948, -0.00...",3
79,2468,False,Hotels & Travel; Event Planning & Services; Ho...,Scottsdale,"6850 E Main St\nScottsdale, AZ 85251",6Kon3cR5ZEm5rmYKlpcfcw,33.493051,-111.933956,Hotel Valley Ho,NaN,...,5,stay one night travel phoenix business cool ho...,review,1,JJnS30Cjv5CBa1JXyZU5yw,4,Hotel,0.336736,"[-0.025511362, 0.049761504, 0.0054899673, -0.0...",0


In [ ]:
restaurant_df['cluster']

,cluster
0,0
1,1
2,0
3,1
4,1
...,...
1743,3
1744,1
1745,0
1746,0


In [ ]:
hotel_df['cluster']

,cluster
0,4
1,1
2,0
3,3
4,3
...,...
76,2
77,0
78,3
79,0


In [ ]:
featuresh = pd.concat([hotel_df['cluster'], hotel_df['DeepMF_predictions'],hotel_df['sentimental score']], axis=1)

In [ ]:
featuresr = pd.concat([restaurant_df['cluster'], restaurant_df['DeepMF_predictions'],restaurant_df['sentimental score']], axis=1)

In [ ]:
labelsh = hotel_df['stars']
labelsr = restaurant_df['stars']

Non Negative Matrix Factorization

In [ ]:
import numpy as np
from sklearn.decomposition import NMF

def nmf(features, n_components=3, init='random', random_state=42):
    nmf = NMF(n_components=n_components, init=init, random_state=random_state)
    nmf_features = nmf.fit_transform(features)
    hybrid_features = np.hstack((features, nmf_features))

    return hybrid_features


In [ ]:
hybrid_faetures_res = nmf(featuresr)
hybrid_faetures_hot = nmf(featuresh)

/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def decision_tree_regressor(features, labels, test_size=0.2, random_state=42):

    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=random_state)

    regressor = DecisionTreeRegressor(random_state=random_state)
    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f'Root Mean Squared Error (RMSE): {rmse}')


In [ ]:
print("Decision Tree Regressor for Restaurants:")
DTR= decision_tree_regressor(hybrid_faetures_res, labelsr)
print("Decision Tree Regressor for Hotels:")
DTH = decision_tree_regressor(hybrid_faetures_hot, labelsh)

Decision Tree Regressor for Restaurants:
Root Mean Squared Error (RMSE): 0.4
Decision Tree Regressor for Hotels:
Root Mean Squared Error (RMSE): 1.1881770515720091


In [ ]:
df.describe()

,Unnamed: 0,business_latitude,business_longitude,business_neighborhoods,business_review_count,business_stars,cool,funny,reviewer_average_stars,reviewer_cool,reviewer_funny,reviewer_review_count,reviewer_useful,stars,useful
count,229907.000000,229907.000000,229907.000000,0.0,229907.000000,229907.000000,229907.000000,229907.000000,229907.000000,229907.000000,229907.000000,229907.000000,229907.000000,229907.000000,229907.000000
mean,114953.000000,33.485976,-111.985018,NaN,113.394199,3.776120,0.868234,0.699030,3.527542,246.295415,200.039177,117.409170,343.676121,3.766723,1.386822
std,66368.578507,0.109553,0.125656,NaN,141.763461,0.594391,1.943242,1.863522,1.066568,915.875702,758.736983,203.242852,1094.918205,1.217010,2.229284
min,0.000000,32.876848,-112.875482,NaN,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,57476.500000,33.422543,-112.066300,NaN,21.000000,3.500000,0.000000,0.000000,3.430000,2.000000,1.000000,9.000000,7.000000,3.000000,0.000000
50%,114953.000000,33.492985,-111.964558,NaN,64.000000,4.000000,0.000000,0.000000,3.750000,15.000000,12.000000,37.000000,42.000000,4.000000,1.000000
75%,172429.500000,33.538927,-111.922262,NaN,145.000000,4.000000,1.000000,1.000000,4.020000,102.000000,79.000000,139.000000,205.000000,5.000000,2.000000
max,229906.000000,34.002867,-111.263508,NaN,862.000000,5.000000,117.000000,70.000000,5.000000,22410.000000,24519.000000,5807.000000,24293.000000,5.000000,120.000000
